In [1]:
import shapely
from shapely import Polygon

from osgeo import gdal
import rasterio

In [2]:
from rasterio.features import rasterize
from rasterstats import zonal_stats

In [3]:
import atlite
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import xarray as xr

In [4]:
from scipy.constants import physical_constants

In [5]:
weather_excel_path = "Parameters/weather_parameters.xlsx"

weather_parameters = pd.read_excel(weather_excel_path,
                                       index_col = 'Parameters'
                                       ).squeeze('columns')
weather_filename = weather_parameters['Filename']

hexagons = gpd.read_file('Resources/hex_transport.geojson')

cutout = atlite.Cutout('Cutouts/' + weather_filename +'.nc')
layout = cutout.uniform_layout()

In [6]:
pv_profile = cutout.pv(
        panel= 'CSi',
        orientation='latitude_optimal',
        layout = layout,
        shapes = hexagons,
        per_unit = True
        )

[########################################] | 100% Completed | 80.81 s


In [74]:
pv_profile

<xarray.DataArray (time: 8784, dim_0: 164)> Size: 12MB
array([[0.        , 0.        , 0.        , ..., 0.00823901, 0.        ,
        0.        ],
       [0.09806071, 0.03523507, 0.11699666, ..., 0.05059279, 0.17309401,
        0.06723595],
       [0.27406973, 0.14849543, 0.33224089, ..., 0.1806548 , 0.35955773,
        0.22924473],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])
Coordinates:
  * time     (time) datetime64[ns] 70kB 2023-01-01 ... 2024-01-01T23:00:00
  * dim_0    (dim_0) int64 1kB 0 1 2 3 4 5 6 7 ... 157 158 159 160 161 162 163
Attributes:
    units:    p.u.

In [7]:
# hexagons_hydro = gpd.read_file('Data/Laos_inc_hydro_hex.geojson')
# fig, ax = plt.subplots(figsize=(10, 10))
# hexagons_hydro.plot(ax=ax, edgecolor='k', cmap='viridis')
hexagons

,h3_index,n0,n1,n2,n3,n4,n5,waterbody_dist,waterway_dist,road_dist,index,theo_turbines,theo_pv,index_right,country,Vientiane road construction costs,Vientiane trucking transport and conversion costs,Vientiane trucking state,Vientiane pipeline transport and conversion costs,geometry
0,846496bffffffff,102,93,38,72,89,110,0.0,0.0,0.0,0,1753,9831,92,Laos,0.0,0.535086,NH3,0.447230,"POLYGON ((101.40997 20.06294, 101.43427 20.310..."
1,8440655ffffffff,19,45,33,9,0,0,0.0,0.0,0.0,1,1767,9547,92,Laos,0.0,0.618337,NH3,0.486964,"POLYGON ((102.28949 21.83502, 102.31457 22.078..."
2,84414d3ffffffff,71,6,40,5,10,78,0.0,0.0,0.0,2,1811,9665,92,Laos,0.0,0.437632,NH3,0.400684,"POLYGON ((102.90898 18.78043, 102.93425 19.028..."
3,8440659ffffffff,27,26,19,108,45,0,0.0,0.0,0.0,3,1740,9430,92,Laos,0.0,0.576556,NH3,0.467007,"POLYGON ((102.21447 21.10103, 102.23944 21.346..."
4,8440659ffffffff,27,26,19,108,45,0,0.0,0.0,0.0,3,1740,9430,139,Other,0.0,0.576556,NH3,0.467007,"POLYGON ((102.21447 21.10103, 102.23944 21.346..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159,84414e1ffffffff,60,47,96,0,0,0,0.0,0.0,0.0,112,1232,6722,92,Laos,0.0,0.553865,NH3,0.456694,"POLYGON ((104.89164 20.05515, 104.91838 20.300..."
160,8441653ffffffff,103,52,25,65,0,0,0.0,0.0,0.0,113,1324,7132,94,Other,0.0,0.667232,NH3,0.510335,"POLYGON ((107.35758 15.61614, 107.38551 15.864..."
161,8441653ffffffff,103,52,25,65,0,0,0.0,0.0,0.0,113,1324,7132,92,Laos,0.0,0.667232,NH3,0.510335,"POLYGON ((107.35758 15.61614, 107.38551 15.864..."
162,84416c3ffffffff,18,90,74,0,0,0,0.0,0.0,0.0,114,1777,5442,92,Laos,0.0,0.552178,NH3,0.455717,"POLYGON ((105.39455 16.23204, 105.42130 16.481..."


In [29]:
layout

<xarray.DataArray (y: 35, x: 30)> Size: 4kB
array([[1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       ...,
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1]])
Coordinates:
  * y        (y) float64 280B 14.0 14.25 14.5 14.75 ... 21.75 22.0 22.25 22.5
  * x        (x) float64 240B 100.2 100.5 100.8 101.0 ... 107.0 107.2 107.5

In [8]:
cutout.data

<xarray.Dataset> Size: 886MB
Dimensions:           (x: 30, y: 35, time: 8784)
Coordinates:
  * x                 (x) float64 240B 100.2 100.5 100.8 ... 107.0 107.2 107.5
  * y                 (y) float64 280B 14.0 14.25 14.5 14.75 ... 22.0 22.25 22.5
  * time              (time) datetime64[ns] 70kB 2023-01-01 ... 2024-01-01T23...
    lon               (x) float64 240B dask.array<chunksize=(30,), meta=np.ndarray>
    lat               (y) float64 280B dask.array<chunksize=(35,), meta=np.ndarray>
Data variables: (12/13)
    height            (y, x) float64 8kB dask.array<chunksize=(35, 30), meta=np.ndarray>
    wnd100m           (time, y, x) float64 74MB dask.array<chunksize=(100, 35, 30), meta=np.ndarray>
    wnd_azimuth       (time, y, x) float64 74MB dask.array<chunksize=(100, 35, 30), meta=np.ndarray>
    roughness         (time, y, x) float64 74MB dask.array<chunksize=(100, 35, 30), meta=np.ndarray>
    influx_toa        (time, y, x) float64 74MB dask.array<chunksize=(100, 35, 30), meta=np.ndarray>
    influx_direct     (time, y, x) float64 74MB dask.array<chunksize=(100, 35, 30), meta=np.ndarray>
    ...                ...
    albedo            (time, y, x) float64 74MB dask.array<chunksize=(100, 35, 30), meta=np.ndarray>
    solar_altitude    (time, y, x) float64 74MB dask.array<chunksize=(100, 35, 30), meta=np.ndarray>
    solar_azimuth     (time, y, x) float64 74MB dask.array<chunksize=(100, 35, 30), meta=np.ndarray>
    temperature       (time, y, x) float64 74MB dask.array<chunksize=(100, 35, 30), meta=np.ndarray>
    soil temperature  (time, y, x) float64 74MB dask.array<chunksize=(100, 35, 30), meta=np.ndarray>
    runoff            (time, y, x) float64 74MB dask.array<chunksize=(100, 35, 30), meta=np.ndarray>
Attributes:
    module:             era5
    prepared_features:  ['height', 'runoff', 'wind', 'influx', 'temperature']
    chunksize_time:     100
    Conventions:        CF-1.6
    history:            2024-04-28 19:27:54 GMT by grib_to_netcdf-2.28.1: /op...

In [26]:
pv_profile.dim_0.data

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163], dtype=int64)

In [25]:
pv_profile.sel(dim_0=100).max()

<xarray.DataArray ()> Size: 8B
array(0.75825574)
Coordinates:
    dim_0    int64 8B 100

In [14]:
location_hydro = gpd.read_file('Data/hydropower_dams.gpkg')
location_hydro.rename(columns={'Latitude': 'lat', 'Longitude': 'lon'}, inplace=True)
location_hydro.rename(columns={'head_example':'head'},inplace=True)
location_hydro

,lat,lon,status,COD,capacity,head,geometry
0,13.943893,105.955782,COMM,2020,260.000,20,POINT (105.95578 13.94389)
1,15.543679,106.260687,COMM,1999,152.100,20,POINT (106.26069 15.54368)
2,15.774865,107.054896,COMM,2023,60.000,20,POINT (107.05490 15.77487)
3,15.354431,106.161330,COMM,2021,5.000,20,POINT (106.16133 15.35443)
4,15.356495,106.498949,COMM,2015,84.800,20,POINT (106.49895 15.35650)
...,...,...,...,...,...,...,...
62,15.491744,106.278621,COMM,1994,45.000,20,POINT (106.27862 15.49174)
63,15.403775,106.280332,COMM,2009,76.000,20,POINT (106.28033 15.40377)
64,20.346323,104.381276,COMM,2019,9.136,20,POINT (104.38128 20.34632)
65,19.977092,103.889435,COMM,2019,32.000,20,POINT (103.88944 19.97709)


In [11]:
location_hydro.capacity.max()

1285.0

In [17]:
laos_hydrobasins = gpd.read_file('hydrobasins_lvl10/hybas_as_lev10_v1c.shp')
laos_hydrobasins['lat'] = location_hydro.geometry.y
laos_hydrobasins['lon'] = location_hydro.geometry.x
laos_hydrobasins.columns

Index(['HYBAS_ID', 'NEXT_DOWN', 'NEXT_SINK', 'MAIN_BAS', 'DIST_SINK',
       'DIST_MAIN', 'SUB_AREA', 'UP_AREA', 'PFAF_ID', 'ENDO', 'COAST', 'ORDER',
       'SORT', 'geometry', 'lat', 'lon'],
      dtype='object')

In [27]:
hexagons

,h3_index,n0,n1,n2,n3,n4,n5,waterbody_dist,waterway_dist,road_dist,hydro,index,geometry
0,85414d9bfffffff,613,142,237,741,583,0,0.000000,0.000000,0.000000,NaN,0,"POLYGON ((103.60514 18.48936, 103.58175 18.581..."
1,85649623fffffff,33,89,102,105,26,515,24.891795,0.000000,0.000000,NaN,1,"POLYGON ((101.09283 20.57548, 101.06831 20.666..."
2,856494d3fffffff,492,470,630,383,28,772,0.000000,0.000000,0.000000,NaN,2,"POLYGON ((101.79826 19.27267, 101.77410 19.364..."
3,85414bd7fffffff,806,689,403,547,335,426,11.131949,0.000000,0.000000,NaN,3,"POLYGON ((103.15971 20.57389, 103.13598 20.665..."
4,85414b6bfffffff,561,53,707,133,144,238,0.000000,0.000000,0.000000,NaN,4,"POLYGON ((102.60469 21.32500, 102.58069 21.415..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
803,8564b06bfffffff,519,301,98,375,766,258,0.000000,11.131949,0.000000,NaN,803,"POLYGON ((101.57960 18.05334, 101.55546 18.145..."
804,854149b3fffffff,530,279,559,328,130,793,0.000000,0.000000,0.000000,NaN,804,"POLYGON ((102.25839 19.22555, 102.23441 19.317..."
805,85414e3bfffffff,538,549,283,191,480,0,11.131949,0.000000,0.000000,9.1,805,"POLYGON ((104.48532 20.26314, 104.46215 20.354..."
806,8541486ffffffff,135,792,771,547,3,689,11.131949,0.000000,11.131949,NaN,806,"POLYGON ((103.28010 20.45408, 103.25643 20.545..."


In [18]:
hydro_profile = cutout.hydro(
    plants=location_hydro,
    hydrobasins= laos_hydrobasins,
    per_unit=True                    # Normalize output per unit area
)

Determine upstream basins per plant: 67it [00:02, 28.04it/s]
c:\Users\lukas\anaconda3\envs\geoh2_master\Lib\site-packages\scipy\sparse\_base.py:730: RuntimeWarning: divide by zero encountered in divide
  recip = np.true_divide(1., other)


[########################################] | 100% Completed | 14.97 s


Shift and aggregate runoff by plant: 67it [00:03, 18.69it/s]


In [19]:
hydro_profile

<xarray.DataArray (plant: 67, time: 8784)> Size: 5MB
array([[2.62211042e+08, 2.66023716e+08, 2.61003815e+08, ...,
        2.64499822e+08, 2.59488087e+08, 2.61326647e+08],
       [4.82567967e+05, 4.83096885e+05, 4.82898397e+05, ...,
        4.71429292e+05, 4.71429292e+05, 4.71429292e+05],
       [2.14390377e+05, 2.16412928e+05, 2.14390377e+05, ...,
        1.44295945e+05, 1.44295945e+05, 1.44295945e+05],
       ...,
       [2.02123996e+05, 1.99300599e+05, 2.00618397e+05, ...,
        2.07845994e+05, 2.09425794e+05, 2.09421183e+05],
       [3.44583073e+05, 3.50646255e+05, 3.69371878e+05, ...,
        3.49532449e+05, 3.65601252e+05, 3.57780938e+05],
       [2.02123996e+05, 1.99300599e+05, 2.00618397e+05, ...,
        2.07845994e+05, 2.09425794e+05, 2.09421183e+05]])
Coordinates:
  * plant    (plant) int64 536B 0 1 2 3 4 5 6 7 8 ... 58 59 60 61 62 63 64 65 66
  * time     (time) datetime64[ns] 70kB 2023-01-01 ... 2024-01-01T23:00:00

In [86]:
hydro_profile.sel(plant=0).min()

<xarray.DataArray ()> Size: 8B
array(42858776.62495468)
Coordinates:
    plant    int64 8B 0

In [62]:
location_hydro.iloc[1,1]

106.260687

In [21]:
hydro_profile.dims[1]

'time'

## Match annual hydroprofile with capacity of hydropower
### 1 Equation hydropower potential

In [32]:
location_hydro

,lat,lon,status,COD,capacity,head,geometry
0,13.943893,105.955782,COMM,2020,260.000,20,POINT (105.95578 13.94389)
1,15.543679,106.260687,COMM,1999,152.100,20,POINT (106.26069 15.54368)
2,15.774865,107.054896,COMM,2023,60.000,20,POINT (107.05490 15.77487)
3,15.354431,106.161330,COMM,2021,5.000,20,POINT (106.16133 15.35443)
4,15.356495,106.498949,COMM,2015,84.800,20,POINT (106.49895 15.35650)
...,...,...,...,...,...,...,...
62,15.491744,106.278621,COMM,1994,45.000,20,POINT (106.27862 15.49174)
63,15.403775,106.280332,COMM,2009,76.000,20,POINT (106.28033 15.40377)
64,20.346323,104.381276,COMM,2019,9.136,20,POINT (104.38128 20.34632)
65,19.977092,103.889435,COMM,2019,32.000,20,POINT (103.88944 19.97709)


In [31]:
hydro_profile

<xarray.DataArray (plant: 67, time: 8784)> Size: 5MB
array([[2.62211042e+08, 2.66023716e+08, 2.61003815e+08, ...,
        2.64499822e+08, 2.59488087e+08, 2.61326647e+08],
       [4.82567967e+05, 4.83096885e+05, 4.82898397e+05, ...,
        4.71429292e+05, 4.71429292e+05, 4.71429292e+05],
       [2.14390377e+05, 2.16412928e+05, 2.14390377e+05, ...,
        1.44295945e+05, 1.44295945e+05, 1.44295945e+05],
       ...,
       [2.02123996e+05, 1.99300599e+05, 2.00618397e+05, ...,
        2.07845994e+05, 2.09425794e+05, 2.09421183e+05],
       [3.44583073e+05, 3.50646255e+05, 3.69371878e+05, ...,
        3.49532449e+05, 3.65601252e+05, 3.57780938e+05],
       [2.02123996e+05, 1.99300599e+05, 2.00618397e+05, ...,
        2.07845994e+05, 2.09425794e+05, 2.09421183e+05]])
Coordinates:
  * plant    (plant) int64 536B 0 1 2 3 4 5 6 7 8 ... 58 59 60 61 62 63 64 65 66
  * time     (time) datetime64[ns] 70kB 2023-01-01 ... 2024-01-01T23:00:00

In [107]:
def hydropower_potential(eta,flowrate,head):
    '''
    Calculate hydropower potential in Megawatts
    eta: Efficiency
    '''
    rho = 997 # kg/m3; Density of water
    g = physical_constants['standard acceleration of gravity'][0] # m/s2; Based on the CODATA constants 2018
    Q = flowrate / 3600 # transform flowrate per h into flowrate per second
    return (eta * rho * g * Q * head) / (1000 * 1000) # MW


eta = 0.75 # Value for system efficiency of hydropower - source
def hydropower_potential_wrapper(flowrate, head):
    return hydropower_potential(eta, flowrate, head)

result = xr.apply_ufunc(
    hydropower_potential_wrapper,
    hydro_profile,
    xr.DataArray(location_hydro['head'], dims=['plant']),  # Ensure head values align with plant dimension
    vectorize=True
)

### 2 Absolute values to capacity factor
#### Change xarray to percentages (If potential > max_capacity: max_capacity ! potential)

In [108]:

# Define the wrapper function to include capacity limiting and capacity factor calculation
def hydropower_potential_with_capacity(flowrate, head, capacity, eta):
    potential = hydropower_potential(flowrate, head, eta)
    limited_potential = xr.where(potential > capacity, capacity, potential)
    capacity_factor = limited_potential / capacity
    return capacity_factor

# Assuming eta is a constant value, flowrate is an xarray.DataArray, and head is a GeoDataFrame
eta = 0.9  # Example efficiency value
head = location_hydro['head'].values  # Extract head values
capacity = location_hydro['capacity'].values  # Extract capacity values

# Create DataArrays for head and capacity to align with the plant dimension
head_da = xr.DataArray(head, dims=['plant'])
capacity_da = xr.DataArray(capacity, dims=['plant'])

# Apply the wrapper function using apply_ufunc
capacity_factor = xr.apply_ufunc(
    hydropower_potential_with_capacity,
    hydro_profile,
    head_da,
    capacity_da,
    eta,
    vectorize=True,
    dask='parallelized',  # Optionally, use Dask for parallel computation
    output_dtypes=[float]
)


In [109]:
capacity_factor

<xarray.DataArray (plant: 67, time: 8784)> Size: 5MB
array([[1.        , 1.        , 1.        , ..., 1.        , 1.        ,
        1.        ],
       [0.15510118, 0.15527118, 0.15520739, ..., 0.15152113, 0.15152113,
        0.15152113],
       [0.17467867, 0.17632658, 0.17467867, ..., 0.11756789, 0.11756789,
        0.11756789],
       ...,
       [1.        , 1.        , 1.        , ..., 1.        , 1.        ,
        1.        ],
       [0.52641687, 0.53567955, 0.56428653, ..., 0.53397799, 0.55852618,
        0.54657915],
       [1.        , 1.        , 1.        , ..., 1.        , 1.        ,
        1.        ]])
Coordinates:
  * plant    (plant) int64 536B 0 1 2 3 4 5 6 7 8 ... 58 59 60 61 62 63 64 65 66
  * time     (time) datetime64[ns] 70kB 2023-01-01 ... 2024-01-01T23:00:00

In [92]:
# Check if all values for each plant are equal to 1 across the time dimension
all_ones_per_plant = (capacity_factor == 1).all(dim='time')

# Count the number of plants that have all values equal to 1
count_all_ones = all_ones_per_plant.sum().item()

# Print the result
print(f"Number of plants with all values equal to 1: {count_all_ones}")


Number of plants with all values equal to 1: 3


### 3 Time-series for each Hexagon
#### Hydro plant capacity factor from plants to hexagons

In [112]:
location_hydro['geometry'] = gpd.points_from_xy(location_hydro.lon, location_hydro.lat)

# Ensure both GeoDataFrames are in the same CRS
if location_hydro.crs != hexagons.crs:
    location_hydro = location_hydro.to_crs(hexagons.crs)

# Perform spatial join to map each hydro plant to its corresponding hexagon
hydro_hex_mapping = gpd.sjoin(location_hydro, hexagons, how='left', predicate='within')

# Add the hydro plant indices to the mapping
hydro_hex_mapping['plant_index'] = hydro_hex_mapping.index

# Get the number of hexagons and time steps dynamically
num_hexagons = len(hexagons)
num_time_steps = len(capacity_factor.time)

# Initialize an empty DataArray for the new capacity_factor with dynamic dimensions
capacity_hexagon = xr.DataArray(
    data=np.zeros((num_hexagons, num_time_steps)),
    dims=['hexagon', 'time'],
    coords={'hexagon': np.arange(num_hexagons), 'time': capacity_factor.time}
)

# Iterate through each hexagon and calculate the average capacity factor for plants within it
for hex_index in range(num_hexagons):
    # Find the plant indices that are within the current hexagon
    plants_in_hex = hydro_hex_mapping[hydro_hex_mapping['index_right'] == hex_index]['plant_index'].tolist()
    
    if len(plants_in_hex) > 0:
        # Select the capacity_factor values for the plants in the current hexagon
        hex_capacity_factor = capacity_factor.sel(plant=plants_in_hex)
        
        # Calculate the average capacity factor for the current hexagon
        average_capacity_factor = hex_capacity_factor.mean(dim='plant')
        
        # Assign the calculated average to the new DataArray
        capacity_hexagon.loc[hex_index] = average_capacity_factor


In [116]:
# Check if all values for each plant are equal to 1 across the time dimension
all_zeros_per_plant = (capacity_hexagon == 1).all(dim='time')

# Count the number of plants that have all values equal to 1
count_all_zeros = all_zeros_per_plant.sum().item()

# Print the result
print(f"Number of plants with all values equal to 1: {count_all_zeros}")


Number of plants with all values equal to 1: 2


In [117]:
capacity_hexagon

<xarray.DataArray (hexagon: 808, time: 8784)> Size: 57MB
array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [1., 1., 1., ..., 1., 1., 1.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])
Coordinates:
  * hexagon  (hexagon) int32 3kB 0 1 2 3 4 5 6 7 ... 801 802 803 804 805 806 807
  * time     (time) datetime64[ns] 70kB 2023-01-01 ... 2024-01-01T23:00:00

## Combined code

In [119]:
def hydropower_potential(eta,flowrate,head):
    '''
    Calculate hydropower potential in Megawatts
    eta: Efficiency
    '''
    rho = 997 # kg/m3; Density of water
    g = physical_constants['standard acceleration of gravity'][0] # m/s2; Based on the CODATA constants 2018
    Q = flowrate / 3600 # transform flowrate per h into flowrate per second
    return (eta * rho * g * Q * head) / (1000 * 1000) # MW

def hydropower_potential_with_capacity(flowrate, head, capacity, eta):
    potential = hydropower_potential(flowrate, head, eta)
    limited_potential = xr.where(potential > capacity, capacity, potential)
    capacity_factor = limited_potential / capacity
    return capacity_factor



In [120]:
head = location_hydro['head'].values  # Extract head values
capacity = location_hydro['capacity'].values  # Extract capacity values

# Create DataArrays for head and capacity to align with the plant dimension
head_da = xr.DataArray(head, dims=['plant'])
capacity_da = xr.DataArray(capacity, dims=['plant'])

# Apply the wrapper function using apply_ufunc
capacity_factor = xr.apply_ufunc(
    hydropower_potential_with_capacity,
    hydro_profile,
    head_da,
    capacity_da,
    eta,
    vectorize=True,
    dask='parallelized',  # Optionally, use Dask for parallel computation
    output_dtypes=[float]
)


In [121]:
location_hydro['geometry'] = gpd.points_from_xy(location_hydro.lon, location_hydro.lat)

# Ensure both GeoDataFrames are in the same CRS
if location_hydro.crs != hexagons.crs:
    location_hydro = location_hydro.to_crs(hexagons.crs)

# Perform spatial join to map each hydro plant to its corresponding hexagon
hydro_hex_mapping = gpd.sjoin(location_hydro, hexagons, how='left', predicate='within')

# Add the hydro plant indices to the mapping
hydro_hex_mapping['plant_index'] = hydro_hex_mapping.index

# Get the number of hexagons and time steps dynamically
num_hexagons = len(hexagons)
num_time_steps = len(capacity_factor.time)

# Initialize an empty DataArray for the new capacity_factor with dynamic dimensions
capacity_hexagon = xr.DataArray(
    data=np.zeros((num_hexagons, num_time_steps)),
    dims=['hexagon', 'time'],
    coords={'hexagon': np.arange(num_hexagons), 'time': capacity_factor.time}
)

# Iterate through each hexagon and calculate the average capacity factor for plants within it
for hex_index in range(num_hexagons):
    # Find the plant indices that are within the current hexagon
    plants_in_hex = hydro_hex_mapping[hydro_hex_mapping['index_right'] == hex_index]['plant_index'].tolist()
    
    if len(plants_in_hex) > 0:
        # Select the capacity_factor values for the plants in the current hexagon
        hex_capacity_factor = capacity_factor.sel(plant=plants_in_hex)
        
        # Calculate the average capacity factor for the current hexagon
        average_capacity_factor = hex_capacity_factor.mean(dim='plant')
        
        # Assign the calculated average to the new DataArray
        capacity_hexagon.loc[hex_index] = average_capacity_factor

In [122]:
capacity_hexagon

<xarray.DataArray (hexagon: 808, time: 8784)> Size: 57MB
array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [1., 1., 1., ..., 1., 1., 1.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])
Coordinates:
  * hexagon  (hexagon) int32 3kB 0 1 2 3 4 5 6 7 ... 801 802 803 804 805 806 807
  * time     (time) datetime64[ns] 70kB 2023-01-01 ... 2024-01-01T23:00:00

In [124]:
# Check if all values for each plant are equal to 1 across the time dimension
all_zeros_per_plant = (capacity_hexagon == 0).all(dim='time')

# Count the number of plants that have all values equal to 1
count_all_zeros = all_zeros_per_plant.sum().item()

# Print the result
print(f"Number of plants with all values equal to 1: {count_all_zeros}")


Number of plants with all values equal to 1: 753
